# Deep Nets with TF Abstractions

Let's explore a few of the various abstractions that TensorFlow offers. You can check out the tf.contrib documentation for more options.

# The Data

To compare these various abstractions we'll use a dataset easily available from the SciKit Learn library. The data is comprised of the results of a chemical analysis of wines grown in the same region in Italy by three different cultivators. There are thirteen different
measurements taken for different constituents found in the three types of wine. We will use the various TF Abstractions to classify the wine to one of the 3 possible labels.

First let's show you how to get the data:

In [1]:
from sklearn.datasets import load_iris
#from sklearn.datasets import load_wine

In [2]:
iris_data = load_iris()

In [3]:
type(iris_data)

sklearn.datasets.base.Bunch

In [4]:
iris_data.keys()

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names'])

In [5]:
print(iris_data.DESCR)

Iris Plants Database

Notes
-----
Data Set Characteristics:
    :Number of Instances: 150 (50 in each of three classes)
    :Number of Attributes: 4 numeric, predictive attributes and the class
    :Attribute Information:
        - sepal length in cm
        - sepal width in cm
        - petal length in cm
        - petal width in cm
        - class:
                - Iris-Setosa
                - Iris-Versicolour
                - Iris-Virginica
    :Summary Statistics:

    ============== ==== ==== ======= ===== ====================
                    Min  Max   Mean    SD   Class Correlation
    ============== ==== ==== ======= ===== ====================
    sepal length:   4.3  7.9   5.84   0.83    0.7826
    sepal width:    2.0  4.4   3.05   0.43   -0.4194
    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
    petal width:    0.1  2.5   1.20  0.76     0.9565  (high!)
    ============== ==== ==== ======= ===== ====================

    :Missing Attribute Values: None
  

In [6]:
feat_data = iris_data['data']

In [7]:
labels = iris_data['target']

### Train Test Split

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(feat_data,
                                                    labels,
                                                    test_size=0.3,
                                                   random_state=101)

### Scale the Data

In [10]:
from sklearn.preprocessing import MinMaxScaler

In [11]:
scaler = MinMaxScaler()

In [12]:
scaled_x_train = scaler.fit_transform(X_train)

In [13]:
scaled_x_test = scaler.transform(X_test)

# Abstractions

## Estimator API

In [14]:
import tensorflow as tf

In [15]:
from tensorflow import estimator 

In [16]:
X_train.shape

(105, 4)

In [17]:
feat_cols = [tf.feature_column.numeric_column("x", shape=[4])]

In [18]:
deep_model = estimator.DNNClassifier(hidden_units=[4,4,4],
                            feature_columns=feat_cols,
                            n_classes=3,
                            optimizer=tf.train.GradientDescentOptimizer(learning_rate=0.01) )

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/7b/2llsq3xs5wv7lqvjqk93v0100000gp/T/tmp2gl0t8wh', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}


In [19]:
input_fn = estimator.inputs.numpy_input_fn(x={'x':scaled_x_train},y=y_train,shuffle=True,batch_size=10,num_epochs=5)

In [20]:
deep_model.train(input_fn=input_fn,steps=500)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/7b/2llsq3xs5wv7lqvjqk93v0100000gp/T/tmp2gl0t8wh/model.ckpt.
INFO:tensorflow:loss = 10.9784, step = 1
INFO:tensorflow:Saving checkpoints for 53 into /var/folders/7b/2llsq3xs5wv7lqvjqk93v0100000gp/T/tmp2gl0t8wh/model.ckpt.
INFO:tensorflow:Loss for final step: 3.35069.


In [21]:
input_fn_eval = estimator.inputs.numpy_input_fn(x={'x':scaled_x_test},shuffle=False)

In [22]:
preds = list(deep_model.predict(input_fn=input_fn_eval))

INFO:tensorflow:Restoring parameters from /var/folders/7b/2llsq3xs5wv7lqvjqk93v0100000gp/T/tmp2gl0t8wh/model.ckpt-53


In [23]:
predictions = [p['class_ids'][0] for p in preds]

In [24]:
from sklearn.metrics import confusion_matrix,classification_report

In [25]:
print(classification_report(y_test,predictions))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00        13
          1       1.00      0.15      0.26        20
          2       0.41      1.00      0.59        12

avg / total       0.84      0.62      0.56        45



____________
______________